# Model

1. data augmentation with grid
1. split train test with uscs clusters
1. predict uscs and sptn
2. first predict uscs, then append it to the table and predict sptn
3. try different models
1. give accuracy vs distance/#boreholes within a region
2. cross validation if necessary

In [3]:
import pandas as pd

In [23]:
df = pd.read_csv('../data/boreholes_uscs_sptn.csv')
df

,Unnamed: 0,Borehole ID,Latitude,Longitude,Elevation (ft),In-situ Test #,Test Elevation (ft),SPTN,Layer #,Layer Top Elevation (ft),Layer Bottom Elevation (ft),Soil Description,USCS,Lithology,Density / Consistency
0,0,21,47.568436,-122.407480,77.30,1,77.30,3.0,1,77.30,75.80,"Grayish brown, dry, very loose, well graded sa...",SW,Fill,Very loose
1,1,21,47.568436,-122.407480,77.30,2,75.80,12.0,2,75.80,70.30,"Clay, medium gray, stiff, slightly moist, with...",CL,Clay,Stiff
2,2,21,47.568436,-122.407480,77.30,3,74.30,8.0,2,75.80,70.30,"Clay, medium gray, stiff, slightly moist, with...",CL,Clay,Medium
3,3,21,47.568436,-122.407480,77.30,4,73.30,7.0,2,75.80,70.30,"Clay, medium gray, stiff, slightly moist, with...",CL,Clay,Medium
4,4,21,47.568436,-122.407480,77.30,5,71.80,7.0,2,75.80,70.30,"Clay, medium gray, stiff, slightly moist, with...",CL,Clay,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19711,19711,153205,47.437412,-122.242456,23.75,9,-8.75,44.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Dense
19712,19712,153205,47.437412,-122.242456,23.75,10,-13.75,23.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Medium
19713,19713,153205,47.437412,-122.242456,23.75,11,-18.75,43.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Dense
19714,19714,153205,47.437412,-122.242456,23.75,12,-23.75,27.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Dense


In [24]:
df['USCS'].unique()

array(['SW', 'CL', 'ML-CL', 'SP', 'SP-ML', 'ML-SP', 'ML', 'SM', 'ML-SM',
       'Not applicable', 'SP-SM', 'OL', nan, 'SM-ML', 'CH', 'SM-SP',
       'SW-SM', 'CL-ML', 'GP', 'OL - SM', 'OL-SM', 'GP-GM', 'GM',
       'Non-standard', 'GW-SW', 'PT', 'GW', 'ML-OL', 'MH', 'SM-OL',
       'SM-GM', 'ML-CH', 'CL-CH', 'OH', 'SP-SW', 'GM-SP', 'SM-CL',
       'ML-MH', 'OH-CL', 'ML-GM', 'GM-SM', 'GM-ML', 'MH-OH', 'SM-SW',
       'GC', 'CL-SM', 'SW-GW', 'OL-ML', 'SP-PT', 'PT-OL', 'SW-PT',
       'SW-SP', 'GM-GP', 'SP-GP', 'CL-SM-CL', 'SC', 'SC-CL', 'SM-SC',
       'ML-SC', 'OL-PT', 'CL-SC', 'OH-PT', 'SM-CH', 'CH-SM', 'SC-SM',
       'GP-SP', 'GW-GM', 'SW-ML', 'SM-GP', 'SP-GW', 'GC-GM', 'OL-SW',
       'GW-GC', 'CH-CL', 'CL-SP', 'ML-PT', 'MH-ML', 'MH-SP', 'MH-SM',
       'ML-OH', 'OH-ML', 'GP-ML', 'SM-PT', 'SP-MH', 'MH-CL', 'SM-GW',
       'GP-SM', 'PT-SP', 'SP-SC', 'Cl', 'AC', 'CR', 'SOD', 'SW-GP',
       'ML\r\nML', 'SP-GM', 'OL\r\nML', 'SW-CL', 'SP-SM, SW-SM', 'PT-CL'],
      dtype=object)

In [25]:
table_uscs = pd.read_csv('../data/file_2_reUSCS_sampling_with_elevation.csv')
table_uscs

,Borehole ID,Document ID,Latitude,Longitude,Elevation,Sampling Depth,Sampling Elevation,USCS
0,1,342,47.574251,-122.407051,148.80,0.600000,148.200000,Fill
1,1,342,47.574251,-122.407051,148.80,1.950000,146.850000,ML
2,1,342,47.574251,-122.407051,148.80,5.350000,143.450000,ML
3,2,342,47.574177,-122.407057,149.50,0.400000,149.100000,Topsoil or vegetation
4,2,342,47.574177,-122.407057,149.50,1.650000,147.850000,SM
...,...,...,...,...,...,...,...,...
391305,153210,66313,47.437303,-122.243141,24.02,22.666667,1.353333,SM
391306,153210,66313,47.437303,-122.243141,24.02,26.333333,-2.313333,SM
391307,153211,66313,47.437099,-122.242644,25.40,4.000000,21.400000,SM
391308,153211,66313,47.437099,-122.242644,25.40,13.000000,12.400000,SM


In [26]:
table_uscs['USCS'].unique()

array(['Fill', 'ML', 'Topsoil or vegetation', 'SM', 'CL', 'CH', 'SC',
       'SW', 'GW', 'SP', 'OH', 'Asphalt or concrete', 'PT', 'OL', 'GM',
       'Bedrock', 'GC', 'GP', 'MH', 'Volcanic ash'], dtype=object)

## Split train-test based on borehole clusters

ie. Cluster some nearby boreholes.
For boreholes that are very close, they can either be in train or in the test.

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split

In [ ]:
# Drop rows with missing coordinates
df_clean = df.dropna(subset=["Latitude", "Longitude", "Test Elevation (ft)"]).copy()

# Compute borehole-level coordinates
borehole_coords = df_clean.groupby("Borehole ID")[["Latitude", "Longitude"]].mean()

# Use DBSCAN to cluster nearby boreholes (eps in degrees, approx. ~0.001 ~ 100m)
dbscan = DBSCAN(eps=0.0015, min_samples=1)
borehole_coords["cluster"] = dbscan.fit_predict(borehole_coords[["Latitude", "Longitude"]])

# Map each borehole to a cluster
borehole_cluster_map = borehole_coords["cluster"].to_dict()
df_clean["cluster"] = df_clean["Borehole ID"].map(borehole_cluster_map)

# Split clusters into train/test
unique_clusters = df_clean["cluster"].unique()
train_clusters, test_clusters = train_test_split(unique_clusters, test_size=0.2, random_state=42)

# Assign train/test set based on cluster
df_clean["set"] = df_clean["cluster"].apply(lambda x: "train" if x in train_clusters else "test")

# Check cluster count and assignments
df_clean["set"].value_counts(), df_clean["cluster"].nunique()


(set
 train    15638
 test      3094
 Name: count, dtype: int64,
 811)

In [13]:
unique_clusters

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [14]:
df_clean

,Unnamed: 0,Borehole ID,Latitude,Longitude,Elevation (ft),In-situ Test #,Test Elevation (ft),SPTN,Layer #,Layer Top Elevation (ft),Layer Bottom Elevation (ft),Soil Description,USCS,Lithology,Density / Consistency,cluster,set
0,0,21,47.568436,-122.407480,77.30,1,77.30,3.0,1,77.30,75.80,"Grayish brown, dry, very loose, well graded sa...",SW,Fill,Very loose,0,train
1,1,21,47.568436,-122.407480,77.30,2,75.80,12.0,2,75.80,70.30,"Clay, medium gray, stiff, slightly moist, with...",CL,Clay,Stiff,0,train
2,2,21,47.568436,-122.407480,77.30,3,74.30,8.0,2,75.80,70.30,"Clay, medium gray, stiff, slightly moist, with...",CL,Clay,Medium,0,train
3,3,21,47.568436,-122.407480,77.30,4,73.30,7.0,2,75.80,70.30,"Clay, medium gray, stiff, slightly moist, with...",CL,Clay,Medium,0,train
4,4,21,47.568436,-122.407480,77.30,5,71.80,7.0,2,75.80,70.30,"Clay, medium gray, stiff, slightly moist, with...",CL,Clay,Medium,0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19711,19711,153205,47.437412,-122.242456,23.75,9,-8.75,44.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Dense,810,train
19712,19712,153205,47.437412,-122.242456,23.75,10,-13.75,23.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Medium,810,train
19713,19713,153205,47.437412,-122.242456,23.75,11,-18.75,43.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Dense,810,train
19714,19714,153205,47.437412,-122.242456,23.75,12,-23.75,27.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Dense,810,train


In [28]:
# Drop rows with missing coordinates
df_clean2 = table_uscs.dropna(subset=["Latitude", "Longitude", "Sampling Elevation"]).copy()

# Compute borehole-level coordinates
borehole_coords2 = df_clean2.groupby("Borehole ID")[["Latitude", "Longitude"]].mean()

# Use DBSCAN to cluster nearby boreholes (eps in degrees, approx. ~0.001 ~ 100m)
dbscan2 = DBSCAN(eps=0.0015, min_samples=1)
borehole_coords2["cluster"] = dbscan.fit_predict(borehole_coords2[["Latitude", "Longitude"]])

# Map each borehole to a cluster
borehole_cluster_map2 = borehole_coords2["cluster"].to_dict()
df_clean2["cluster"] = df_clean2["Borehole ID"].map(borehole_cluster_map2)

# Split clusters into train/test
unique_clusters2 = df_clean2["cluster"].unique()
train_clusters2, test_clusters2 = train_test_split(unique_clusters2, test_size=0.2, random_state=42)

# Assign train/test set based on cluster
df_clean2["set"] = df_clean2["cluster"].apply(lambda x: "train" if x in train_clusters2 else "test")

# Check cluster count and assignments
df_clean2["set"].value_counts(), df_clean2["cluster"].nunique()


(set
 train    345511
 test      44573
 Name: count, dtype: int64,
 5066)

In [29]:
unique_clusters2[-1]

np.int64(5065)

## Train

### Random forest

Here, we use df to train and test both uscs and sptn

In [ ]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_absolute_error

In [21]:
df_uscs['USCS'].unique()

array(['SW', 'CL', 'ML-CL', 'SP', 'SP-ML', 'ML-SP', 'ML', 'SM', 'ML-SM',
       'Not applicable', 'SP-SM', 'OL', 'SM-ML', 'CH', 'SM-SP', 'SW-SM',
       'CL-ML', 'GP', 'OL - SM', 'OL-SM', 'GP-GM', 'GM', 'Non-standard',
       'GW-SW', 'PT', 'GW', 'ML-OL', 'MH', 'SM-OL', 'SM-GM', 'ML-CH',
       'CL-CH', 'OH', 'SP-SW', 'GM-SP', 'SM-CL', 'ML-MH', 'OH-CL',
       'ML-GM', 'GM-SM', 'GM-ML', 'MH-OH', 'SM-SW', 'GC', 'CL-SM',
       'SW-GW', 'OL-ML', 'SP-PT', 'PT-OL', 'SW-PT', 'SW-SP', 'GM-GP',
       'SP-GP', 'CL-SM-CL', 'SC', 'SC-CL', 'SM-SC', 'ML-SC', 'OL-PT',
       'CL-SC', 'OH-PT', 'SM-CH', 'CH-SM', 'SC-SM', 'GP-SP', 'GW-GM',
       'SW-ML', 'SM-GP', 'SP-GW', 'GC-GM', 'OL-SW', 'GW-GC', 'CH-CL',
       'CL-SP', 'ML-PT', 'MH-ML', 'MH-SP', 'MH-SM', 'ML-OH', 'OH-ML',
       'GP-ML', 'SM-PT', 'SP-MH', 'MH-CL', 'SM-GW', 'GP-SM', 'PT-SP',
       'SP-SC', 'Cl', 'AC', 'CR', 'SOD', 'SP-GM', 'OL\r\nML', 'SW-CL',
       'SP-SM, SW-SM', 'SW-GP', 'PT-CL'], dtype=object)

In [20]:
len(df_uscs['USCS'].unique())

98

In [18]:
# Separate datasets for classification (USCS) and regression (SPTN)
df_uscs = df_clean.dropna(subset=['USCS']).copy()
df_sptn = df_clean.dropna(subset=['SPTN']).copy()

# Encode the USCS categorical target variable
df_uscs['USCS_encoded'] = df_uscs['USCS'].astype('category').cat.codes

# Define features
features = ['Test Elevation (ft)', 'Latitude', 'Longitude']

# Classification: USCS
X_train_uscs = df_uscs[df_uscs['set'] == 'train'][features]
y_train_uscs = df_uscs[df_uscs['set'] == 'train']['USCS_encoded']
X_test_uscs = df_uscs[df_uscs['set'] == 'test'][features]
y_test_uscs = df_uscs[df_uscs['set'] == 'test']['USCS_encoded']

# Regression: SPTN
X_train_sptn = df_sptn[df_sptn['set'] == 'train'][features]
y_train_sptn = df_sptn[df_sptn['set'] == 'train']['SPTN']
X_test_sptn = df_sptn[df_sptn['set'] == 'test'][features]
y_test_sptn = df_sptn[df_sptn['set'] == 'test']['SPTN']

# Train Random Forest Classifier for USCS
clf_uscs = RandomForestClassifier(n_estimators=100, random_state=42)
clf_uscs.fit(X_train_uscs, y_train_uscs)
y_pred_uscs = clf_uscs.predict(X_test_uscs)
accuracy_uscs = accuracy_score(y_test_uscs, y_pred_uscs)

# Train Random Forest Regressor for SPTN
reg_sptn = RandomForestRegressor(n_estimators=100, random_state=42)
reg_sptn.fit(X_train_sptn, y_train_sptn)
y_pred_sptn = reg_sptn.predict(X_test_sptn)
mae_sptn = mean_absolute_error(y_test_sptn, y_pred_sptn)

accuracy_uscs, mae_sptn

(0.2345803842264914, 21.8105008336668)

Here we first predict uscs on table_uscs

In [30]:
table_uscs['USCS'].unique()

array(['Fill', 'ML', 'Topsoil or vegetation', 'SM', 'CL', 'CH', 'SC',
       'SW', 'GW', 'SP', 'OH', 'Asphalt or concrete', 'PT', 'OL', 'GM',
       'Bedrock', 'GC', 'GP', 'MH', 'Volcanic ash'], dtype=object)

In [32]:
# Separate datasets for classification (USCS) and regression (SPTN)
df_uscs = df_clean2.dropna(subset=['USCS']).copy()

# Encode the USCS categorical target variable
df_uscs['USCS_encoded'] = df_uscs['USCS'].astype('category').cat.codes

# Define features
features = ['Sampling Elevation', 'Latitude', 'Longitude']

# Classification: USCS
X_train_uscs = df_uscs[df_uscs['set'] == 'train'][features]
y_train_uscs = df_uscs[df_uscs['set'] == 'train']['USCS_encoded']
X_test_uscs = df_uscs[df_uscs['set'] == 'test'][features]
y_test_uscs = df_uscs[df_uscs['set'] == 'test']['USCS_encoded']

# Train Random Forest Classifier for USCS
clf_uscs = RandomForestClassifier(n_estimators=100, random_state=42)
clf_uscs.fit(X_train_uscs, y_train_uscs)
y_pred_uscs = clf_uscs.predict(X_test_uscs)
accuracy_uscs = accuracy_score(y_test_uscs, y_pred_uscs)

accuracy_uscs

0.41390527898054874

### Gradiant Boosting

In [33]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Separate datasets for classification (USCS)
df_uscs = df_clean2.dropna(subset=['USCS']).copy()

# Encode the USCS categorical target variable
df_uscs['USCS_encoded'] = df_uscs['USCS'].astype('category').cat.codes

# Define features
features = ['Sampling Elevation', 'Latitude', 'Longitude']

# Classification: USCS
X_train_uscs = df_uscs[df_uscs['set'] == 'train'][features]
y_train_uscs = df_uscs[df_uscs['set'] == 'train']['USCS_encoded']
X_test_uscs = df_uscs[df_uscs['set'] == 'test'][features]
y_test_uscs = df_uscs[df_uscs['set'] == 'test']['USCS_encoded']

# Train Gradient Boosting Classifier for USCS
clf_gb = GradientBoostingClassifier(n_estimators=50, max_depth=3, random_state=42)
clf_gb.fit(X_train_uscs, y_train_uscs)

# Predict and evaluate the accuracy
y_pred_uscs_gb = clf_gb.predict(X_test_uscs)
accuracy_uscs_gb = accuracy_score(y_test_uscs, y_pred_uscs_gb)

accuracy_uscs_gb


0.32068741166176834

### Support Vector Machines (SVM)

### Neural Networks (MLPClassifier)